In [3]:
"""
import sys
reload(sys)
sys.setdefaultencoding('utf8')
"""
from gensim.corpora.wikicorpus import extract_pages,filter_wiki
import bz2file
import re
import opencc
from tqdm import tqdm
import codecs

In [4]:
wiki = extract_pages(bz2file.open('./zhwiki-latest-pages-articles.xml.bz2'))

In [8]:
cc = opencc.OpenCC('t2s')

def wiki_replace(d):
    s = d[1]
    s = re.sub(':*{\|[\s\S]*?\|}', '', s)
    s = re.sub('<gallery>[\s\S]*?</gallery>', '', s)
    s = re.sub('(.){{([^{}\n]*?\|[^{}\n]*?)}}', '\\1[[\\2]]', s)
    s = filter_wiki(s)
    s = re.sub('\* *\n|\'{2,}', '', s)
    s = re.sub('\n+', '\n', s)
    s = re.sub('\n[:;]|\n +', '\n', s)
    s = re.sub('\n==', '\n\n==', s)
    s = u'【' + d[0] + u'】\n' + s
    return cc.convert(s).strip()
    # return s.strip()
 
i = 0
f = codecs.open('wiki.txt', 'w', encoding='utf-8')
w = tqdm(wiki, desc=u'已获取0篇文章')
for d in w:
    if not re.findall('^[a-zA-Z]+:', d[0]) and d[0] and not re.findall(u'^#', d[1]):
        s = wiki_replace(d)
        f.write(s+'\n\n\n')
        i += 1
        if i % 100 == 0:
            w.set_description(u'已获取%s篇文章'%i)
 
f.close()

已获取1143800篇文章: : 3558026it [1:25:26, 693.98it/s]


In [11]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from gensim.models import word2vec
import logging, jieba
import os, io

words_file = "./wiki.txt" 
words = list()
i = 0 
with io.open(words_file, 'r', encoding="utf-8") as words_file_object: 
  contents = words_file_object.readlines() 
  for line in contents: 
    print (line)
    i += 1
    if i == 200:
      break

【哲学】



苏格拉底之死，由雅克·路易·大卫所绘（1787年）

哲学（philosophy）是研究普遍的、基本问题的学科，包括存在、知识、价值、理智、心灵、语言等领域。哲学与其他学科不同之处在于哲学有独特之思考方式，例如批判的方式、通常是系统化的方法，并以理性论证为基础。在日常用语中，哲学可被引申为个人或团体的最基本信仰、概念或态度，不过此处并非此定义。



== 简介 ==



=== 词源 ===

英语词语Philosophy（philosophia）源于古希腊语中的φιλοσοφία，意思为「爱智慧」，有时也译为「智慧的朋友」，该词由φίλος（philos，爱）的派生词φιλεῖν（Philein，去爱）和σοφία（Sophia，智慧）组合而成。一般认为，古希腊思想家毕达哥拉斯最先在著作中引入“哲学家”和“哲学”这两个术语。

“哲”一词在中国起源很早，如“孔门十哲”，“古圣先哲”等词，“哲”或“哲人”，专指那些善于思辨，学问精深者，即西方近世“哲学家”，“思想家”之谓。在《易经》当中已经开始讨论哲学问题，形而上学的中文名称取自《易经·系辞上传》「形而上者谓之道，形而下者谓之器」一语。1874年，日本启蒙家西周，在《百一新论》中首先用汉文「哲学」来翻译philosophy一词。



=== 哲学的定义 ===

英国哲学家罗素对哲学的定义是：哲学，就我对这个词的理解来说，乃是某种介乎神学与科学之间的东西。它和神学一样，包含着人类对于那些迄今仍为科学知识所不能肯定之事物的思考；但它又像科学一样，是诉之于人类的理性而不是诉之于权威的，不论是传统的权威还是启示的权威。一切确切的知识（罗素认为）都属于科学；一切涉及超乎确切知识之外的教条都属于神学。但介乎神学与科学之间还有一片受到双方攻击的无人之域，这片无人之域就是哲学。

胡适在《中国哲学史大纲》中称「凡研究人生切要的问题，从根本上着想，要寻一个根本的解决：这种学问叫做哲学」。

虽然哲学源自西方的传统，但许多文明在历史上都存在著一些相似的论题。东亚和南亚的哲学被称之为东方哲学，而北非和中东则因为其和欧洲密切的互动，因此常被视为是西方哲学的一部份。

对哲学的主题亦存在许多看法。一些人认为哲学是对问题本身过程的审查；另外一些人则认为实质上存在著哲学必须去回答的哲学命题。

*后现代主义把哲学定义为创

In [16]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from gensim.models import word2vec
import logging, jieba
import os, io


logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO) 
sentences = word2vec.Text8Corpus(u"./wiki.txt")
if os.path.exists("./wiki_model"): 
  model = word2vec.Word2Vec.load('./wiki_model') 
else: 
  #这里迭代次数iter默认为5，我使用的是100，生成size=100维词向量，出现少于min_count=5次的词忽略
  model = word2vec.Word2Vec(sentences, min_count=5, size=100, iter=100)
  model.save("./wiki_model")

#下面是测试生成的模型
y = model.most_similar(u'男孩', topn = 10)
for item in y:
  print('和男孩最相似的是： ', item[0], item[1])
print('--------------------------------------')
y = model.most_similar(u'足球', topn = 10)
for item in y:
  print('和足球最相似的是： ', item[0], item[1])
print('--------------------------------------')
print(model[u'早餐'])

: INFO : EPOCH 98 - PROGRESS: at 4.24% examples, 318211 words/s, in_qsize 0, out_qsize 0
2020-09-12 19:49:59,440 : INFO : EPOCH 98 - PROGRESS: at 6.38% examples, 323575 words/s, in_qsize 0, out_qsize 0
2020-09-12 19:50:00,443 : INFO : EPOCH 98 - PROGRESS: at 8.34% examples, 322174 words/s, in_qsize 0, out_qsize 0
2020-09-12 19:50:01,448 : INFO : EPOCH 98 - PROGRESS: at 9.74% examples, 302622 words/s, in_qsize 0, out_qsize 0
2020-09-12 19:50:02,452 : INFO : EPOCH 98 - PROGRESS: at 10.77% examples, 280573 words/s, in_qsize 0, out_qsize 0
2020-09-12 19:50:03,462 : INFO : EPOCH 98 - PROGRESS: at 12.02% examples, 268300 words/s, in_qsize 0, out_qsize 0
2020-09-12 19:50:04,467 : INFO : EPOCH 98 - PROGRESS: at 13.65% examples, 268051 words/s, in_qsize 0, out_qsize 0
2020-09-12 19:50:05,473 : INFO : EPOCH 98 - PROGRESS: at 15.56% examples, 273532 words/s, in_qsize 0, out_qsize 0
2020-09-12 19:50:06,475 : INFO : EPOCH 98 - PROGRESS: at 17.74% examples, 285457 words/s, in_qsize 0, out_qsize 0
20

In [21]:
y = model.most_similar(u'建筑', topn = 10)
for item in y:
  print('最相似的是： ', item[0], item[1])

最相似的是：  图片 0.748155415058136
最相似的是：  建筑设计 0.7148329615592957
最相似的是：  建筑风格 0.7062318921089172
最相似的是：  建筑特点 0.6815007925033569
最相似的是：  画廊 0.6688799858093262
最相似的是：  特色 0.66258704662323
最相似的是：  建筑特色 0.6541027426719666
最相似的是：  主要建筑 0.6501036882400513
最相似的是：  馆藏 0.6418026685714722
最相似的是：  藏品 0.6412068605422974
D:\Anaconda3\envs\python36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [27]:
model.similarity(u'物联网', u'通信')

D:\Anaconda3\envs\python36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


0.34998924